# MSc Thesis 
# Jacotte Monroe

Code that retrieves automatically the MODIS and Landsat scenes of interest and loads them. 

## Importing and initializing 

In [31]:
import ee
import geemap
import os

ee.Initialize()

## Set-up

In [6]:
# define dates 
start_date = '2008-10-01' # '2014-02-15'
end_date = '2014-10-01' # '2012-02-17'

In [2]:
# load Etosha National Park study area
enp = ee.FeatureCollection('WCMC/WDPA/current/polygons') \
        .filter(ee.Filter.eq('ORIG_NAME', 'Etosha'))

# turn ENP study area into a geometry
enp_geom = enp.geometry()

In [3]:
# define sample study area 
# note: sample area is smaller than original sample area because of Landsat data size limit when exporting 
sample_area = ee.Geometry.Polygon(
        [[[15.457215604955591, -18.68807168881119],
          [15.457215604955591, -19.120709491695354],
          [15.939240751439966, -19.120709491695354],
          [15.939240751439966, -18.68807168881119]]], None)

In [2]:
# import more packages --> to put at top when cleaning code 
import pandas as pd

In [34]:
# load extent look up table 
extents_lut = pd.read_csv('data/step_extents/LA2_step_ex_w2027.csv')

(7, 7)


In [36]:
# get first geometry
e1 = extents_lut.iloc[0]

# retrieve extent coordinates 
# source: https://sparkbyexamples.com/pandas/pandas-get-cell-value-from-dataframe/?utm_content=cmp-true
# source: https://stackoverflow.com/questions/75203044/how-to-create-polygon-from-bbox-data-in-pythonv
e1_coords = [[[e1.loc['xmin'], e1.loc['ymin']],
              [e1.loc['xmin'], e1.loc['ymax']],
              [e1.loc['xmax'], e1.loc['ymax']],
              [e1.loc['xmax'], e1.loc['ymin']]]]

# generate new bbox geometry 
bbox = ee.Geometry.Polygon(e1_coords, None)

{'type': 'Polygon', 'coordinates': [[[16.0135379637369, -19.3372499859631], [16.0740598660107, -19.3372499859631], [16.0740598660107, -19.2395934386144], [16.0135379637369, -19.2395934386144], [16.0135379637369, -19.3372499859631]]]}


In [42]:
# get date of geometry
e1_start_date = e1.loc['start_date']
e1_end_date = e1.loc['end_date']

## Dataset retrieval

In [7]:
## Landsat 8 
# note: ee.List should also normally include 180 (removed it for this sampled study area)
l8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterDate(start_date, end_date) \
    .filter(ee.Filter.inList('WRS_PATH', ee.List([179]))) \
    .filter(ee.Filter.eq('WRS_ROW', 73)) \
    .select(['SR_B.'])

# remove B1 from Landsat 8 image collection 
# source: https://developers.google.com/earth-engine/apidocs/ee-list-remove
l8 = l8.select(ee.List(l8.first().bandNames().getInfo()).remove('SR_B1'))

# retrieve L8 dates 
# source: https://developers.google.com/earth-engine/apidocs/ee-imagecollection-aggregate_array
l8_dates = l8.aggregate_array('DATE_ACQUIRED')
# source: https://gis.stackexchange.com/questions/307115/earth-engine-get-dates-from-imagecollection
# source (python API): https://developers.google.com/earth-engine/guides/python_install#syntax
l8_dates = l8_dates.map(lambda x: ee.Date(x).advance(1, 'day').format('YYYY_MM_dd'))

#print('L8 dates', l8_dates.getInfo())

In [4]:
## MODIS daily 250m
#alternative to filterDate: filter(ee.Filter.inList('system:index', l8_dates)) \
modis = ee.ImageCollection('MODIS/061/MOD09GQ') \
    .filterDate('2014-01-14', '2014-02-18') \
    .filterBounds(sample_area) \
    .select(['sur_refl_b02', 'sur_refl_b01']) 

## Functions

In [8]:
# function to clip image to study area
def clipToAOI(image): 
    result = image.clip(sample_area)
    return result.copyProperties(image, ['system:id'])

In [9]:
# define landsat scaling factor
def applyScaleFactors(image): 
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    return image.addBands(opticalBands, None, True)

In [10]:
# function to reproject (32733 if want Namibia coordinates, but elephant fixes are in 4326)
def reprojectLandsat(image): 
    return image.reproject('EPSG:4326', None, 30)

def reprojectModis(image):
    return image.reproject('EPSG:4326', None, 250)


## Apply functions

In [11]:
# apply reprojection 
modis_reproj = modis.map(reprojectModis)
l8_reproj = l8.map(reprojectLandsat)

In [12]:
# apply clipping function 
modis_clipped = modis_reproj.map(clipToAOI)
l8_clipped = l8_reproj.map(clipToAOI)

In [13]:
# apply scaling factor 
l8_scaled = l8_clipped.map(applyScaleFactors)

# is this function actually working? because when i map this it doesn't match visualization parameters 
    # for scaled image instead it matches for non scaled image

## Visualize

In [44]:
# define visualization parameters
visualization_modis = {
  'min': -100.0,
  'max': 8000.0,
  'bands': ['sur_refl_b02', 'sur_refl_b02', 'sur_refl_b01'],
}

visualization_l8 = {
  'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
  'min': 8000,
  'max': 20000,
}

visualization_l8_scaled = {
  'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
  'min': 0,
  'max': 0.5,
}

In [60]:
# initialization and set up of the map
Map = geemap.Map()
#Map.add_ee_layer(enp, None, 'Etosha National Park')
Map.add_layer(sample_area, None, 'Sample AOI')
Map.add_layer(l8_scaled.first(), visualization_l8_scaled, 'L8 Scaled')
Map.centerObject(enp_geom)

# Show the map
#Map

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.

## Export data

In [14]:
# get output folder path 
# source: https://github.com/gee-community/geemap/blob/master/examples/notebooks/11_export_image.ipynb
out_dir_l8 = os.path.join(os.path.expanduser('~'), 'Documents/MSc_thesis/data/l8')
out_dir_m = os.path.join(os.path.expanduser('~'), 'Documents/MSc_thesis/data/modis')

In [18]:
# export Landsat image collection to local repository 
# source: https://github.com/gee-community/geemap/blob/master/examples/notebooks/11_export_image.ipynb
geemap.ee_export_image_collection(l8_clipped, out_dir = out_dir_l8)

Total number of images: 32

Exporting 1/32: /home/osboxes/Documents/MSc_thesis/data/l8/LC08_179073_20130418.tif
Generating URL ...
Please wait ...
Data downloaded to /home/osboxes/Documents/MSc_thesis/data/l8/LC08_179073_20130418.tif


Exporting 2/32: /home/osboxes/Documents/MSc_thesis/data/l8/LC08_179073_20130504.tif
Generating URL ...
Please wait ...
Data downloaded to /home/osboxes/Documents/MSc_thesis/data/l8/LC08_179073_20130504.tif


Exporting 3/32: /home/osboxes/Documents/MSc_thesis/data/l8/LC08_179073_20130520.tif
Generating URL ...
Please wait ...
Data downloaded to /home/osboxes/Documents/MSc_thesis/data/l8/LC08_179073_20130520.tif


Exporting 4/32: /home/osboxes/Documents/MSc_thesis/data/l8/LC08_179073_20130605.tif
Generating URL ...
Please wait ...
Data downloaded to /home/osboxes/Documents/MSc_thesis/data/l8/LC08_179073_20130605.tif


Exporting 5/32: /home/osboxes/Documents/MSc_thesis/data/l8/LC08_179073_20130621.tif
Generating URL ...
Please wait ...
Data downloaded to /h

Data downloaded to /home/osboxes/Documents/MSc_thesis/data/l8/LC08_179073_20140421.tif


Exporting 23/32: /home/osboxes/Documents/MSc_thesis/data/l8/LC08_179073_20140507.tif
Generating URL ...
Please wait ...
Data downloaded to /home/osboxes/Documents/MSc_thesis/data/l8/LC08_179073_20140507.tif


Exporting 24/32: /home/osboxes/Documents/MSc_thesis/data/l8/LC08_179073_20140523.tif
Generating URL ...
Please wait ...
Data downloaded to /home/osboxes/Documents/MSc_thesis/data/l8/LC08_179073_20140523.tif


Exporting 25/32: /home/osboxes/Documents/MSc_thesis/data/l8/LC08_179073_20140608.tif
Generating URL ...
Please wait ...
Data downloaded to /home/osboxes/Documents/MSc_thesis/data/l8/LC08_179073_20140608.tif


Exporting 26/32: /home/osboxes/Documents/MSc_thesis/data/l8/LC08_179073_20140624.tif
Generating URL ...
Please wait ...
Data downloaded to /home/osboxes/Documents/MSc_thesis/data/l8/LC08_179073_20140624.tif


Exporting 27/32: /home/osboxes/Documents/MSc_thesis/data/l8/LC08_179073_201

In [15]:
# export MODIS image collection to local repository 
# source: https://github.com/gee-community/geemap/blob/master/examples/notebooks/11_export_image.ipynb
geemap.ee_export_image_collection(modis_clipped, out_dir = out_dir_m)

Total number of images: 35

Exporting 1/35: /home/osboxes/Documents/MSc_thesis/data/modis/2014_01_14.tif
Generating URL ...
Please wait ...
Data downloaded to /home/osboxes/Documents/MSc_thesis/data/modis/2014_01_14.tif


Exporting 2/35: /home/osboxes/Documents/MSc_thesis/data/modis/2014_01_15.tif
Generating URL ...
Please wait ...
Data downloaded to /home/osboxes/Documents/MSc_thesis/data/modis/2014_01_15.tif


Exporting 3/35: /home/osboxes/Documents/MSc_thesis/data/modis/2014_01_16.tif
Generating URL ...
Please wait ...
Data downloaded to /home/osboxes/Documents/MSc_thesis/data/modis/2014_01_16.tif


Exporting 4/35: /home/osboxes/Documents/MSc_thesis/data/modis/2014_01_17.tif
Generating URL ...
Please wait ...
Data downloaded to /home/osboxes/Documents/MSc_thesis/data/modis/2014_01_17.tif


Exporting 5/35: /home/osboxes/Documents/MSc_thesis/data/modis/2014_01_18.tif
Generating URL ...
Please wait ...
Data downloaded to /home/osboxes/Documents/MSc_thesis/data/modis/2014_01_18.tif


Ex

Please wait ...
Data downloaded to /home/osboxes/Documents/MSc_thesis/data/modis/2014_02_05.tif


Exporting 24/35: /home/osboxes/Documents/MSc_thesis/data/modis/2014_02_06.tif
Generating URL ...
Please wait ...
Data downloaded to /home/osboxes/Documents/MSc_thesis/data/modis/2014_02_06.tif


Exporting 25/35: /home/osboxes/Documents/MSc_thesis/data/modis/2014_02_07.tif
Generating URL ...
Please wait ...
Data downloaded to /home/osboxes/Documents/MSc_thesis/data/modis/2014_02_07.tif


Exporting 26/35: /home/osboxes/Documents/MSc_thesis/data/modis/2014_02_08.tif
Generating URL ...
Please wait ...
Data downloaded to /home/osboxes/Documents/MSc_thesis/data/modis/2014_02_08.tif


Exporting 27/35: /home/osboxes/Documents/MSc_thesis/data/modis/2014_02_09.tif
Generating URL ...
Please wait ...
Data downloaded to /home/osboxes/Documents/MSc_thesis/data/modis/2014_02_09.tif


Exporting 28/35: /home/osboxes/Documents/MSc_thesis/data/modis/2014_02_10.tif
Generating URL ...
Please wait ...
Data down